In [1]:
from kiutra_api.api_client import KiutraClient

In [ ]:
# Initiate RPCClient to communicate with "kiutra base module"
host = '192.168.11.20'
client = KiutraClient(host, 1006)

In [ ]:
client.query('cryostat.status')

In [ ]:
api_info = client.query('api_manager.api_info')

In [ ]:
for device in api_info:
    print(device)
    print('------------------------------------------------------------------')
    try:
        api_functions = client.query(f"{device}.api")
        functions = list(api_functions.keys())
        for func in functions:
            #print(api_functions[func])
            try:
                print(f'{func} query:\n'+api_functions[func]['output_doc'])
                print(f'{func} set:\n'+api_functions[func]['input_doc'])
            except:
                try:
                    print(f'{func} call:\n' + api_functions[func]['doc'])
                except:
                    pass
    except Exception as e:
        print(e)
        pass
    print('------------------------------------------------------------------')
    print('\n')

In [ ]:
print(client.query('api_manager.access_level'))

In [ ]:
client.query('T_adr1.kelvin')

In [ ]:
client.set('adr_control.pre_regenerate', True)

In [ ]:
print(client.query('T_adr1.kelvin_buffer'))
print(client.query('T_adr1.kelvin_rate'))
print(client.query('T_adr1.kelvin_median'))
print(client.query('T_adr1.kelvin_timestamp'))
print(client.query('T_adr1.kelvin'))

In [ ]:
client.query('adr_control.api')

In [ ]:
client.call('adr_control.start_adr', setpoint=1.0, ramp=0.1, adr_mode=2, operation_mode="cadr", auto_regenerate=False, pre_regenerate=True)

In [ ]:
client.query('T_adr1.kelvin_rate')

In [ ]:
client.call('cryostat.device_check')

# Measurement Example

In [ ]:
def spinning_cursor():
    """
    Spinning cursor to show activity
    """
    while True:
        for cursor in '|/-\\':
            yield cursor

In [ ]:
def refresh_line(content):
    print(content, end='\r')

In [ ]:
from kiutra_api.controller_interfaces import ContinuousTemperatureControl, MagnetControl, ADRControl
import time

spinner = spinning_cursor()

host = '192.168.11.20'
T_start = 0.2
T_end = 6
T_ramp = 0.3

start_field = 0
fields = [0.5, 2.0, 3.0]
field_ramp = 0.5


temperature_control = ContinuousTemperatureControl('temperature_control', host)
sample_magnet = MagnetControl('sample_magnet', host)
adr_control = ADRControl('adr_control', host)
code, message = temperature_control.status
temperature_control.reset()
adr_control.recharge()


T = temperature_control.kelvin
B = sample_magnet.field

print(f"Initializing: T = {T} K, B = {B} T")

# prepare adr system for magnetic cooling
# this is only necessary when the initial setpoint is below bath (~4 K) or to maximize cooling power/hold-time when ramping up and down
# if only heating is required, this step can be skipped
print(f"Waiting for ADR system to be ready")
while adr_control.is_active:
    T = temperature_control.kelvin
    B = sample_magnet.field
    refresh_line(f'{next(spinner)} - T = {T:.3f} K, B = {B:.3f} {"T":<2}')
    time.sleep(1)
print("")

# starting field-heated, zero-field cooled
for field in fields:
    print(f"Starting field-heated, zero-field cooled sweep from {T_start} K to {T_end} K @ {T_ramp} K/min with a field of {field} T")
    # 1. ramp magnetic field to start_field
    sample_magnet.start((start_field, field_ramp))
    print(f"Waiting for {start_field} T ")
    while True:
        T = temperature_control.kelvin
        B = sample_magnet.field

        stable = sample_magnet.stable
        _, status = sample_magnet.status
        refresh_line(f'{next(spinner)} - T = {T:.3f} K, B = {B:.3f} {"T":<2} ({status}, stable={stable})')
        if stable:
            break
        time.sleep(1)
    print("")


    T = temperature_control.kelvin
    B = sample_magnet.field

    print(f"Magnet ready: T = {T} K, B = {B} T")

    # 2. go to start temperature
    temperature_control.start((T_start, T_ramp))
    print(f"Waiting for {T_start} K ")
    while True:
        ramping_info = temperature_control.get_ramping_info()

        T = temperature_control.kelvin
        B = sample_magnet.field
        refresh_line(f'{next(spinner)} - T = {T:.3f} K, B = {B:.3f} {"T":<2}')
        if ramping_info['ramp_done'] and ramping_info['ready_to_ramp']:
            # when whole ramp is covered, go on
            break
        time.sleep(1)
    print("")

    T = temperature_control.kelvin
    B = sample_magnet.field

    print(f"Temperature ready: T = {T} K, B = {B} T")

    # 3. ramp to target field
    sample_magnet.start((field, field_ramp))
    print(f"Waiting for {field} T ")
    while True:
        T = temperature_control.kelvin
        B = sample_magnet.field

        stable = sample_magnet.stable
        _, status = sample_magnet.status
        refresh_line(f'{next(spinner)} - T = {T:.3f} K, B = {B:.3f} {"T":<2} ({status}, stable={stable})')
        if stable:
            break
        time.sleep(1)
    print("")

    T = temperature_control.kelvin
    B = sample_magnet.field

    print(f"Magnet done: T = {T} K, B = {B} T")

    # 4. ramp to target temperature
    temperature_control.start((T_end, T_ramp))
    print(f"Waiting for {T_end} K ")
    while True:
        ramping_info = temperature_control.get_ramping_info()
        T = temperature_control.kelvin
        B = sample_magnet.field
        refresh_line(f'{next(spinner)} - T = {T:.3f} K, B = {B:.3f} {"T":<2}')
        if ramping_info['ramp_done'] and ramping_info['ready_to_ramp'] and ramping_info['stable']: # some of the checks are redundant
            # when whole ramp is covered, go on
            break
        time.sleep(1)
    print("")

    T = temperature_control.kelvin
    B = sample_magnet.field

    print(f"Temperature done: T = {T} K, B = {B} T")
    print(f"Finished sweep from {T_start} K to {T_end} K @ {T_ramp} K/min with a field of {field} T")
    # repeat
print("--------------------------> done")

# temperature_control.start_sequence([(0.1, 0.15, 120), (5.0, 0.2, 0)])

